## Prediccion de Precios de Viviendas

El mercado inmobiliario es uno de los sectores donde el ML tiene mayor aplicación práctica. Los agentes inmobiliarios, inversores y compradores necesitan herramientas para estimar valores de propiedades basándose en características como ubicación, tamaño, antigüedad, etc.

### Entrenamiento del Modelo de Predicción de Precios de Casas

In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

### Generar dataset sintético (500+ registros)

In [30]:
np.random.seed(42)
n_samples = 600

### Crear datos sintéticos realistas

In [31]:
data = {
    'area_m2': np.random.normal(120, 40, n_samples),
    'habitaciones': np.random.choice([1, 2, 3, 4, 5], n_samples, p=[0.1, 0.2, 0.4, 0.2, 0.1]),
    'baños': np.random.choice([1, 2, 3, 4], n_samples, p=[0.3, 0.4, 0.2, 0.1]),
    'antiguedad': np.random.randint(0, 50, n_samples),
    'distancia_centro': np.random.exponential(10, n_samples),
    'garaje': np.random.choice([0, 1], n_samples, p=[0.3, 0.7]),
    'jardin': np.random.choice([0, 1], n_samples, p=[0.6, 0.4])
}

df = pd.DataFrame(data)

### Crear precio basado en características (fórmula realista)

In [32]:
precio_base = (
    df['area_m2'] * 1500 +  # $1500 por m2
    df['habitaciones'] * 8000 +  # $8000 por habitación
    df['baños'] * 5000 +  # $5000 por baño
    (50 - df['antiguedad']) * 500 +  # Depreciación por antigüedad
    (20 - df['distancia_centro']) * 1000 +  # Proximidad al centro
    df['garaje'] * 15000 +  # Bonus por garaje
    df['jardin'] * 10000  # Bonus por jardín
)

### Añadir ruido realista

In [33]:
ruido = np.random.normal(0, 10000, n_samples)
df['precio'] = np.maximum(precio_base + ruido, 30000)  # Precio mínimo

In [34]:
print("ANÁLISIS EXPLORATORIO")
print(f"Dataset shape: {df.shape}")
print(f"Registros: {len(df)}")
print("\nPrimeras 5 filas:")
print(df.head())

print("\nEstadísticas descriptivas:")
print(df.describe())

ANÁLISIS EXPLORATORIO
Dataset shape: (600, 8)
Registros: 600

Primeras 5 filas:
      area_m2  habitaciones  baños  antiguedad  distancia_centro  garaje  \
0  139.868566             2      2          41          5.482407       1   
1  114.469428             4      2          34          3.496152       1   
2  145.907542             4      2          45          6.655330       1   
3  180.921194             3      2          25          8.608558       1   
4  110.633865             3      1           4          0.447245       1   

   jardin         precio  
0       1  280811.825718  
1       0  244790.870277  
2       0  305888.386091  
3       0  328385.049060  
4       1  248614.471144  

Estadísticas descriptivas:
          area_m2  habitaciones       baños  antiguedad  distancia_centro  \
count  600.000000    600.000000  600.000000  600.000000        600.000000   
mean   119.459229      2.983333    2.088333   24.650000         10.095163   
std     38.887659      1.123310    0.91756

### División del dataset

In [35]:
X = df.drop('precio', axis=1)
y = df['precio']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nConjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Conjunto de prueba: {X_test.shape[0]} muestras")


Conjunto de entrenamiento: 480 muestras
Conjunto de prueba: 120 muestras


### Entrenamiento del modelo

In [36]:
print("\nENTRENAMIENTO DEL MODELO")
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)


ENTRENAMIENTO DEL MODELO


,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


### Predicciones

In [37]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

### Métricas

In [38]:
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("\nMÉTRICAS DEL MODELO")
print(f"MAE Entrenamiento: ${train_mae:,.2f}")
print(f"MAE Prueba: ${test_mae:,.2f}")
print(f"R² Entrenamiento: {train_r2:.4f}")
print(f"R² Prueba: {test_r2:.4f}")
print(f"RMSE Prueba: ${test_rmse:,.2f}")


MÉTRICAS DEL MODELO
MAE Entrenamiento: $4,799.32
MAE Prueba: $12,493.37
R² Entrenamiento: 0.9905
R² Prueba: 0.9279
RMSE Prueba: $16,640.68


### Importancia de características

In [39]:
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nImportancia de características:")
print(feature_importance)


Importancia de características:
            feature  importance
0           area_m2    0.904195
4  distancia_centro    0.042032
3        antiguedad    0.022575
1      habitaciones    0.016324
2             baños    0.006265
5            garaje    0.006114
6            jardin    0.002494


### Guardar el modelo y scaler

In [40]:
print("\nGUARDANDO MODELO")
joblib.dump(model, 'house_price_model.pkl')
joblib.dump(X.columns.tolist(), 'feature_names.pkl')


GUARDANDO MODELO


['feature_names.pkl']

### Guardar métricas para la API

In [41]:
metrics = {
    'mae_test': float(test_mae),
    'r2_test': float(test_r2),
    'rmse_test': float(test_rmse),
    'n_samples': len(df),
    'features': X.columns.tolist()
}

import json
with open('model_metrics.json', 'w') as f:
    json.dump(metrics, f)

print("Modelo guardado como 'house_price_model.pkl'")
print("Métricas guardadas como 'model_metrics.json'")
print("\nENTRENAMIENTO COMPLETADO")

Modelo guardado como 'house_price_model.pkl'
Métricas guardadas como 'model_metrics.json'

ENTRENAMIENTO COMPLETADO


### Ejemplo de predicción

In [42]:
print("\nEJEMPLO DE PREDICCIÓN")
ejemplo = [[100, 3, 2, 5, 8, 1, 1]]  # [area, hab, baños, antigüedad, dist_centro, garaje, jardín]
prediccion = model.predict(ejemplo)
print(f"Casa ejemplo - Precio predicho: ${prediccion[0]:,.2f}")


EJEMPLO DE PREDICCIÓN
Casa ejemplo - Precio predicho: $225,207.10


c:\Users\IPF-2025\Desktop\PRIMER CUATRIMESTRE\TLP3 - Python para Ciencia de Datos\house-price-predictor\env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
